In [0]:
import uuid
#importamos la funcion lit
from pyspark.sql.functions import lit

# Variables de rutas y metadatos
entidad = "empleados"
source_file = "workspace.fuentes_csv.datos_empleados_20251028"
target_file = "workspace.capa_bronze.stg1_datos_empleados"
source_system = "hr_system"
pipeline_run_id = str(uuid.uuid4())

# Etapa Bronze: leer la tabla original
df_bronze = spark.table(source_file)

#Agragamos metadatos tecnicos al df_bronze
df_bronze = df_bronze.withColumn("source_system", lit(source_system)) \
                     .withColumn("pipeline_run_id", lit(pipeline_run_id)) \
                     .withColumn("entity", lit(entidad)) \
                     .withColumn("source_file", lit(source_file)) \
                     .withColumn("target_file", lit(target_file))

# Guardar el DataFrame como tabla Delta en la etapa Bronze (Medallón)
df_bronze.write.format("delta").mode("overwrite").saveAsTable(target_file)

# Leer la tabla Bronze
bronze_table_df = spark.table(target_file)

# Contar registros
num_registros = bronze_table_df.count()
print(f"Cantidad de registros en Bronze: {num_registros}")

df_bronze.describe()